<a href="https://colab.research.google.com/github/AnastasiaBrinati/Progetto-ML-23-24/blob/main/task3_centralizzato.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import libraries


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.layers import LeakyReLU, PReLU
import numpy as np
from keras.initializers import RandomNormal, he_normal, glorot_normal, he_uniform, glorot_uniform
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
!pip install -q -U keras-tuner
import keras_tuner as kt
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, PReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.initializers import he_normal, glorot_normal

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 2.4 MB/s eta 0:00:00


# Dataset loading and preprocessing

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from keras.datasets import cifar10

# Carica il dataset CIFAR-10
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# Normalizzazione
X_train = X_train / 255.0
X_test = X_test / 255.0

170498071/170498071 [==============================] - 4s 0us/step


# Hyperparameters tuning

In [3]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, AveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import kerastuner as kt

def model_builder(hp):
    model = Sequential()

    # Tune the number of convolutional layers
    for i in range(hp.Int('num_conv_layers', 1, 3)):
        hp_filters = hp.Int(f'filters_{i}', min_value=16, max_value=128, step=16)
        hp_kernel_size = hp.Choice(f'kernel_size_{i}', values=[3, 5])
        hp_pooling = hp.Choice(f'pooling_{i}', values=['max', 'avg'])

        model.add(Conv2D(filters=hp_filters, kernel_size=hp_kernel_size, activation='relu', padding='same'))

        if hp_pooling == 'max':
            model.add(MaxPooling2D(pool_size=(2, 2)))
        else:
            model.add(AveragePooling2D(pool_size=(2, 2)))

    model.add(Flatten())

    # Tune the number of units and activation function for each Dense layer
    for i in range(hp.Int('num_dense_layers', 1, 3)):
        hp_units = hp.Int(f'units_{i}', min_value=16, max_value=128, step=16)
        hp_activation = hp.Choice(f'activation_{i}', values=['PReLU','LeakyReLU','relu', 'tanh', 'sigmoid'])

        model.add(Dense(units=hp_units, activation=hp_activation))

    # Output layer
    num_classes = 26  # Numero di classi per il dataset EMNIST Letters
    model.add(Dense(num_classes, activation='softmax'))

    # Tune the learning rate for the optimizer
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

    model.compile(optimizer=Adam(learning_rate=hp_learning_rate),
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    return model

tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='multi_layer_tuning')

stop_early = EarlyStopping(monitor='val_loss', patience=5)

tuner.search(X_train
             , y_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]

# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(X_train, y_train, epochs=30, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch:', best_epoch)

# Retrain the model
hypermodel = tuner.hypermodel.build(best_hps)
hypermodel.fit(X_train, y_train, epochs=best_epoch, validation_split=0.2)

# Evaluate the model on the test set
eval_result = hypermodel.evaluate(X_test, y_test)
print("Test loss:", eval_result[0])
print("Test accuracy:", eval_result[1])


<ipython-input-3-2aab99a20cfa>:4: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt



Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
3                 |3                 |num_conv_layers
112               |112               |filters_0
3                 |3                 |kernel_size_0
max               |max               |pooling_0
3                 |3                 |num_dense_layers
112               |112               |units_0
relu              |relu              |activation_0
0.01              |0.01              |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
2                 |2                 |tuner/bracket
0                 |0                 |tuner/round

Epoch 1/2
 418/1250 [=========>....................] - ETA: 1:23 - loss: 2.3510 - accuracy: 0.0967

KeyboardInterrupt: 

In [ ]:
hypermodel.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_9 (Conv2D)           (None, 28, 28, 112)       2912      
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 14, 14, 112)       0         
 g2D)                                                            
                                                                 
 conv2d_10 (Conv2D)          (None, 14, 14, 48)        48432     
                                                                 
 average_pooling2d_6 (Avera  (None, 7, 7, 48)          0         
 gePooling2D)                                                    
                                                                 
 conv2d_11 (Conv2D)          (None, 7, 7, 128)         55424     
                                                                 
 average_pooling2d_7 (Avera  (None, 3, 3, 128)        

In [ ]:
best_hps.values

{'num_conv_layers': 3,
 'filters_0': 112,
 'kernel_size_0': 5,
 'pooling_0': 'max',
 'num_dense_layers': 2,
 'units_0': 64,
 'activation_0': 'PReLU',
 'learning_rate': 0.001,
 'filters_1': 48,
 'kernel_size_1': 3,
 'pooling_1': 'avg',
 'units_1': 64,
 'activation_1': 'PReLU',
 'units_2': 128,
 'activation_2': 'tanh',
 'filters_2': 128,
 'kernel_size_2': 3,
 'pooling_2': 'avg',
 'tuner/epochs': 10,
 'tuner/initial_epoch': 4,
 'tuner/bracket': 2,
 'tuner/round': 2,
 'tuner/trial_id': '0012'}